In [1]:

import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix
)
from xgboost import XGBClassifier, XGBRegressor
import joblib

In [5]:
DATA_DIR = Path(r"D:\HOCTAP\YDCC\EarthEngine\EarthEngine")

csv_files = sorted(DATA_DIR.glob("flood_dataset_dbscl_*.csv"))
if not csv_files:
    raise FileNotFoundError(" Không tìm thấy file CSV dataset")

df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

print(" Dataset loaded")
print("Shape:", df.shape)
print(df.head())


 Dataset loaded
Shape: (168000, 8)
                system:index        VV  elevation  flood  month     slope  \
0  1_1_1_1_1_1_1_1_1_1_1_2_0 -7.263228          3      0      1  1.322575   
1  1_1_1_1_1_1_1_1_1_1_1_2_1 -6.181550          3      0      1  1.321602   
2  1_1_1_1_1_1_1_1_1_1_1_2_2 -8.386385          2      0      1  2.096116   
3  1_1_1_1_1_1_1_1_1_1_1_2_3 -3.951998          2      0      1  1.323290   
4  1_1_1_1_1_1_1_1_1_1_1_2_4 -9.877078          3      0      1  1.322802   

   year                                    .geo  
0  2018  {"type":"MultiPoint","coordinates":[]}  
1  2018  {"type":"MultiPoint","coordinates":[]}  
2  2018  {"type":"MultiPoint","coordinates":[]}  
3  2018  {"type":"MultiPoint","coordinates":[]}  
4  2018  {"type":"MultiPoint","coordinates":[]}  


In [6]:
required_cols = ["VV", "elevation", "slope", "flood", "year"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f" Missing columns: {missing}")

print("Flood ratio:")
print(df["flood"].value_counts(normalize=True))


Flood ratio:
flood
0    0.913589
1    0.086411
Name: proportion, dtype: float64


#### CHỌN FEATURE & TARGET

In [7]:
features = ["VV", "elevation", "slope"]
target = "flood"

X = df[features]
y = df[target]

#### TRAIN / TEST SPLIT THEO THỜI GIAN

In [8]:
train_df = df[df["year"] <= 2022]
test_df  = df[df["year"] > 2022]

Xtr = train_df[features]
ytr = train_df[target]

Xte = test_df[features]
yte = test_df[target]

print("Train size:", Xtr.shape)
print("Test size :", Xte.shape)

Train size: (120000, 3)
Test size : (48000, 3)


#### HANDLE IMBALANCE

In [9]:
pos_weight = (ytr == 0).sum() / (ytr == 1).sum()
print("scale_pos_weight =", round(pos_weight, 2))

scale_pos_weight = 10.55


#### TRAIN XGBOOST

In [10]:
model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=pos_weight,
    eval_metric="auc",
    random_state=42,
    n_jobs=-1
)

model.fit(Xtr, ytr)

print("✅ Model training completed")

✅ Model training completed


#### ĐÁNH GIÁ MODEL

In [11]:
from sklearn.metrics import roc_auc_score, classification_report

proba = model.predict_proba(Xte)[:, 1]

auc = roc_auc_score(yte, proba)
print("\nAUC =", round(auc, 4))

print("\nClassification report (threshold = 0.3):")
print(classification_report(yte, (proba > 0.3).astype(int)))


AUC = 1.0

Classification report (threshold = 0.3):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     43869
           1       0.97      1.00      0.98      4131

    accuracy                           1.00     48000
   macro avg       0.99      1.00      0.99     48000
weighted avg       1.00      1.00      1.00     48000



#### FEATURE IMPORTANCE

In [12]:
imp = pd.Series(model.feature_importances_, index=features)
imp = imp.sort_values(ascending=False)

print("\nFeature importance:")
print(imp)


Feature importance:
VV           0.987492
slope        0.008354
elevation    0.004154
dtype: float32


#### LƯU MODEL (CHO REALTIME / API)

In [13]:
MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True)

joblib.dump(model, MODEL_DIR / "flood_xgb_classifier.pkl")

print("💾 Model saved to models/flood_xgb_classifier.pkl")

💾 Model saved to models/flood_xgb_classifier.pkl
